In [1]:
%%capture
%pip install llama-index llama-index-embeddings-openai qdrant-client llama-index-vector-stores-qdrant llama-index llama-index-llms-openai llama-index-vector-stores-faiss faiss-cpu llama-index-llms-anthropic tavily-python

In [2]:
import os
import nest_asyncio
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(r"C:\Users\anteb\PycharmProjects\JupyterProject\.env")
nest_asyncio.apply()

CO_API_KEY = os.environ.get('CO_API_KEY') or getpass("Enter CO_API_KEY: ")
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or getpass("Enter OPENAI_API_KEY: ")
QDRANT_URL = os.environ.get('QDRANT_URL') or getpass("Enter QDRANT_URL: ")
QDRANT_API_KEY = os.environ.get('QDRANT_API_KEY') or getpass("Enter QDRANT_API_KEY: ")
TAVILY_API_KEY = os.environ.get('TAVILY_API_KEY') or getpass("Enter TAVILY_API_KEY: ")

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.5, max_tokens=512)


In [4]:
from tavily import AsyncTavilyClient

# note the type annotations for the incoming query and the return string
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))

In [5]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    system_prompt="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information.",
)

In [27]:
response = await workflow.run(user_msg="What is the weather in Smolensk Russia?")
print(str(response))

The current weather in Smolensk, Russia, is as follows:

- **Temperature**: 7.7°C (45.9°F)
- **Condition**: Sunny
- **Wind**: 4.3 mph (6.8 kph) from the northwest
- **Humidity**: 70%
- **Visibility**: 10 km
- **Pressure**: 1017 mb

For more details, you can check the weather on [WeatherAPI](https://www.weatherapi.com/) or [AccuWeather](https://www.accuweather.com/en/ru/smolensk/295475/weather-forecast/295475).


In [8]:
from llama_index.core.workflow import Context

# configure a context to work with our workflow
ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Daniil, nice to meet you!", ctx=ctx # give the configured context to the workflow
)
print(str(response))

Nice to meet you, Daniil! How can I assist you today?


In [9]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

# convert our Context to a dictionary object
ctx_dict = ctx.to_dict(serializer=JsonSerializer())

# create a new Context from the dictionary
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [10]:
response = await workflow.run(
    user_msg="Do you still remember my name?", ctx=restored_ctx
)
print(str(response))

Yes, I remember your name, Daniil! How can I help you today?


In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Smolenks? Call me by name when you anser", ctx=restored_ctx)


async for event in handler.stream_events():
    if isinstance(event, AgentInput):
       print("Agent input: ", event.input)  # the current input messages
       print("Agent name:", event.current_agent_name)  # the current agent name
    elif isinstance(event, AgentOutput):
       print("Agent output: ", event.response)  # the current full response
       print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
       print("Raw LLM response: ", event.raw)  # the raw llm api response
    elif isinstance(event, ToolCallResult):
       print("Tool called: ", event.tool_name)  # the tool name
       print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
       print("Tool output: ", event.tool_output)  # the tool output

## Tools and State

Tools can also be defined that have access to the workflow context. This means you can set and retrieve variables from the context and use them in the tool or between tools.

`AgentWorkflow` uses a context variable called `state` that gets passed to every agent. You can rely on information in `state` being available without explicitly having to pass it in.

**Note:** To access the `Context`, the Context parameter should be the first parameter of the tool.

In [31]:
from llama_index.core.workflow import Context

async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"


workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

response = await workflow.run(user_msg="My name is Daniil", ctx=ctx)
print(str(response))

state = await ctx.get("state")
print("Name as stored in state: ",state["name"])

Your name has been set to Daniil.
Name as stored in state:  Daniil


## Human in the Loop

Tools can also be defined that involve a human in the loop. This is useful for tasks that require human input, such as confirming a tool call or providing feedback.

As we'll see in the next section, the way `Workflow`s work under the hood of `AgentWorkflow` is by running `step`s which both emit and receive events. Here's a diagram of the steps (in blue) that makes up an AgentWorkflow and the events (in green) that pass data between them. You'll recognize these events, they're the same ones we were handling in the output stream earlier.

 To get a human in the loop, we'll get our tool to emit an event that isn't received by any other step in the workflow. We'll then tell our tool to wait until it receives a specific "reply" event.

This is a very common pattern for human in the loop, so we have built-in `InputRequiredEvent` and `HumanResponseEvent` events to use for this purpose. If you want to capture different forms of human input, you can subclass these events to match your own preferences.

In [6]:
from llama_index.core.workflow import (
    Context,
    InputRequiredEvent,
    HumanResponseEvent,
)

# a tool that performs a dangerous task
async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""

    # emit an event to the external stream to be captured
    ctx.write_event_to_stream(
        InputRequiredEvent(
            prefix="Are you sure you want to proceed? ",
            user_name="Daniil",
        )
    )

    # wait until we see a HumanResponseEvent
    response = await ctx.wait_for_event(
        HumanResponseEvent, requirements={"user_name": "Daniil"}
    )

    # act on the input from the event
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."


workflow = AgentWorkflow.from_tools_or_functions(
    [dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)

To capture the event, we use the same streaming interface we used earlier and look for an `InputRequiredEvent`. Then we can use `input` to capture a response from the user, and send it back using the `send_event` method.

In [8]:
handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
    # capture InputRequiredEvent
    if isinstance(event, InputRequiredEvent):
        # capture keyboard input
        response = input(event.prefix)
        # send our response back
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response,
                user_name=event.user_name,
            )
        )

response = await handler
print(str(response))

The dangerous task has been aborted. If you wish to proceed, please confirm that you are ready to continue with the task.


## Putting together a multi-agent system

Now we've seen all the ways workflows can work, let's construct our own multi-agent system!

We'll have to create new agents, since the FunctionCallingAgents we made earlier are specifically adapted to run inside of `AgentWorkflow`. Making a generic function calling agent is very similar:

In [9]:
from llama_index.core.agent import FunctionCallingAgent as GenericFunctionCallingAgent
from llama_index.core.tools import FunctionTool

# convert our web search functions into a tool, the generic function calling agent doesn't do this automatically (yet)
search_web_tool = FunctionTool.from_defaults(fn=search_web)

research_agent = GenericFunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that does research by searching the web and then records the results of your research."
)
write_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that writes a report based on the results of research by another agent."
)
review_agent = GenericFunctionCallingAgent.from_tools(
    tools=[],
    llm=llm,
    verbose=False,
    allow_parallel_tool_calls=False,
    system_prompt="You are an agent that reviews a report written by a different agent."
)

We'll create a simple linear flow to start with:

In [10]:
from llama_index.core.workflow import Event, Workflow, Context, StopEvent, step
from llama_index.core.workflow import StartEvent

class ResearchEvent(Event):
    prompt: str

class WriteEvent(Event):
    research: str

class ReviewEvent(Event):
    report: str

class ReviewResults(Event):
    review: str

class SimpleMultiAgentFlow(Workflow):

    @step
    async def setup(self, ev: StartEvent) -> ResearchEvent:
        self.research_agent = ev.research_agent
        self.write_agent = ev.write_agent
        self.review_agent = ev.review_agent
        return ResearchEvent(prompt=ev.prompt)

    @step
    async def research(self, ctx: Context, ev: ResearchEvent) -> WriteEvent:

        await ctx.set("prompt", ev.prompt)
        result = self.research_agent.chat(f"Do some research that another agent will use to write a report about this topic: <topic>{ev.prompt}</topic>. Just include the facts without making it into a full report.")

        return WriteEvent(research=str(result))

    @step
    async def write(self, ctx: Context, ev: WriteEvent) -> ReviewEvent:
        result = self.write_agent.chat(f"Write a report based on this research: <research>{ev.research}</research> and this topic: <topic>{await ctx.get('prompt')}</topic>")

        return ReviewEvent(report=str(result))

    @step
    async def review(self, ctx: Context, ev: ReviewEvent) -> StopEvent:
        result = self.review_agent.chat(f"Review this report: <report>{ev.report}</report>")

        ctx.write_event_to_stream(ReviewResults(review=str(result)))

        return StopEvent(result=ev.report)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

workflow = SimpleMultiAgentFlow(timeout=30, verbose=True)
handler = workflow.run(
    prompt="History of San Francisco",
    research_agent=research_agent,
    write_agent=write_agent,
    review_agent=review_agent
)

async for ev in handler.stream_events():
    if isinstance(ev, ReviewResults):
        print("==== The review ====")
        print(ev.review)

final_result = await handler
print("==== The report ====")
print(final_result)

## Adding Reflection

But we mentioned reflection as an extra feature we can add because we have fine-grained control. Let's add that!

We need to make several changes:
* In `research` we'll store the research into the context, since we might need to use it multiple times
* We'll tell `write` that it can be triggered by a `RewriteEvent` in addition to a `WriteEvent`
* If it's a `RewriteEvent` we'll add the review as feedback to the prompt
* `review` will be changed to optionally emit a `RewriteEvent`
* We'll get the LLM to decide if the review returned by the agent is a "bad" or "good" review

In [12]:
class RewriteEvent(Event):
    review: str

# Our WriteEvent doesn't need research attached any more
class WriteEvent(Event):
    pass

class MultiAgentFlowWithReflection(Workflow):

    @step
    async def setup(self, ev: StartEvent) -> ResearchEvent:
        self.research_agent = ev.research_agent
        self.write_agent = ev.write_agent
        self.review_agent = ev.review_agent
        return ResearchEvent(prompt=ev.prompt)

    @step
    async def research(self, ctx: Context, ev: ResearchEvent) -> WriteEvent:

        await ctx.set("prompt", ev.prompt)
        result = self.research_agent.chat(f"Do some research that another agent will use to write a report about this topic: <topic>{ev.prompt}</topic>. Just include the facts without making it into a full report.")

        # store the research in the context for multiple uses
        await ctx.set("research", str(result))

        return WriteEvent()

    @step
    async def write(self, ctx: Context, ev: WriteEvent | RewriteEvent ) -> ReviewEvent:

        prompt = f"Write a report based on this research: <research>{await ctx.get('research')}</research> and this topic: <topic>{await ctx.get('prompt')}</topic> "

        # detect RewriteEvents and modify the prompt
        if isinstance(ev, RewriteEvent):
            print("Doing a rewrite!")
            prompt += f"This report has reviewed and the reviewer had this feedback, which you should take into account: <review>{ev.review}</review>"

        result = self.write_agent.chat(prompt)

        return ReviewEvent(report=str(result))

    @step
    async def review(self, ctx: Context, ev: ReviewEvent) -> StopEvent | RewriteEvent:
        result = self.review_agent.chat(f"Review this report: {ev.report}")

        # get the LLM to self-reflect
        try_again = llm.complete(f"This is a review of a report written by an agent. If you think this review is bad enough that the agent should try again, respond with just the word RETRY. If the review is good, reply with just the word CONTINUE. Here's the review: <review>{str(result)}</review>")

        if try_again.text == "RETRY":
            print("Reviewer said try again")
            return RewriteEvent(review=str(result))
        else:
            print("Reviewer thought it was good!")
            return StopEvent(result=ev.report)

In [ ]:
workflow = MultiAgentFlowWithReflection(timeout=30, verbose=True)
handler = workflow.run(
    prompt="History Smolensk, Russia",
    research_agent=research_agent,
    write_agent=write_agent,
    review_agent=review_agent
)

final_result = await handler
print("==== The report ====")
print(final_result)